In [1]:
import geocoder
import pandas as pd
import numpy as np
import datetime
import time
import logging
from haversine import haversine
# logging.basicConfig(filename="celery"+ str(datetime.datetime.now().date())+".log", level=logging.INFO)

In [29]:
rewards_data=pd.read_table("/home/jian/Projects/Smoothie_King/data/LoyaltyUser_103018.txt",dtype=str,sep=",")

In [30]:
rewards_data_df_TA=rewards_data[['user_id','zipcode','latitude','longitude']]
rewards_data_df_TA['zipcode']=rewards_data_df_TA['zipcode'].fillna("0")
rewards_data_df_TA['zipcode']=rewards_data_df_TA['zipcode'].apply(lambda x: x.split("-")[0].zfill(5))
rewards_data_df_TA['latitude']=rewards_data_df_TA['latitude'].astype(float).apply(lambda x: np.round(x,6))
rewards_data_df_TA['longitude']=rewards_data_df_TA['longitude'].astype(float).apply(lambda x: np.round(x,6))

# Round to 6 digits which result in error about 0.14 meter

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [31]:
zips_TA=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/SmoothieKing_TA_revised_3_miles_JL_2018-10-31.xlsx",dtype=str,sheetname="zip_TA")
store_TA=pd.read_excel("/home/jian/Projects/Smoothie_King/TA/SmoothieKing_TA_revised_3_miles_JL_2018-10-31.xlsx",dtype=str,sheetname="output_TA_by_store")
store_TA=store_TA[store_TA['status']!="NonComp"]
zips_TA=zips_TA[zips_TA['store'].isin(store_TA['storenumber'])]
zips_in_TA_list=zips_TA['zip'].unique().tolist()
stores_in_TA_list=store_TA['storenumber'].unique().tolist()

# Step 1: Use zipcode in the rewards data to check in

In [32]:
rewards_data_df_TA_in=rewards_data_df_TA[(rewards_data_df_TA['zipcode'].isin(zips_in_TA_list))]
rewards_data_df_TA_in['TA']="In"
rewards_data_df_TA_out=rewards_data_df_TA[~(rewards_data_df_TA['zipcode'].isin(zips_in_TA_list))]
rewards_data_df_TA_out['TA']="Out"
unique_lat_lng_pair_in_out=rewards_data_df_TA_out[['latitude','longitude']].drop_duplicates()

# Step 2: For those zipcodes not in, check lat lng

In [33]:
unique_lat_lng_pair_in_out=rewards_data_df_TA_out[['latitude','longitude']].drop_duplicates()
unique_lat_lng_pair_in_out=unique_lat_lng_pair_in_out.reset_index()
del unique_lat_lng_pair_in_out['index']
unique_lat_lng_pair_in_out['Bing_zip']=np.nan
unique_lat_lng_pair_in_out['Bing_lat']=np.nan
unique_lat_lng_pair_in_out['Bing_lng']=np.nan
unique_lat_lng_pair_in_out['Bing_return_dist']=np.nan
unique_lat_lng_pair_in_out['Bing_Address']=np.nan

In [34]:
key_9="AufGjbgbDfkYqR_I1MR9z_CRFBt9S5gwvMldoxxH5nNUuVU7J1UCfThKiCNvZfme"
key_10="AjJtraMlHbaR3sDKcFqFud61zsa6Psu8uISUqf8yuNmTLoMdvxI0RLvKnfH21E9N"
key_11="AsE84xQndNKNRWmVoFtPArCHx9vbNj-_Azb__FVPOUQKAsJRkLka9OGaHUW9Jg1H"
key_12="AlldgEkiRHdNvlvYi75IYtknSEEmP_F5lRK10dlFcIsE8zm35rEPYOFi7vntfa9m"
key_13="AmManQR4nORBLY98XKAU6DT0koWFTnSPGc9IKuG116L2rEz5g2wSeKSa_wt_PiVs"
key_14="Anf0V62fjg15L7RmpiXy4tpUbrue9Jgsls_9cGKs9ZEllJ5ctMKtreVUrnikOJYK"
key_15="Av7T0zTuritczWO2HASvXyIZsSx2WuFCcUbVbBOh4s2CjgCYzy6LNlpgVGPWFIu1"
key_16="Aglce2Z_CkofvgQE4EVugK08EmqKX0K3sHoRq2HDbG27GTr5rpxABilJdg95gdjL"

key_list=[key_9,key_10,key_11,key_12,key_13,key_14,key_15,key_16]
# All tested to be workable
print(str(datetime.datetime.now())+": Start to run Bing Map")
output_lat_long=pd.DataFrame()

# Can be optimised later through "batch_reverse" method

2018-11-05 16:41:13.591414: Start to run Bing Map


In [36]:
i=0

df=unique_lat_lng_pair_in_out.iloc[i*100000:(i+1)*100000,].reset_index()
key_bing=key_list[i]
del df['index']
for j in range(len(df)):
    coordinate_pair=[df['latitude'][j],df['longitude'][j]]
    try:
        response=geocoder.bing(coordinate_pair,key=key_bing,method="reverse")
    except:
        response=[]


    try:
        bing_lat=response.latlng[0]
    except:
        bing_lat=np.nan

    try:
        bing_lng=response.latlng[1]
    except:
        bing_lng=np.nan

    try:
        bing_address=response.address
    except:
        bing_address=np.nan

    try:
        bing_postal=response.postal
    except:
        bing_postal=np.nan    

    try:
        dist=haversine(coordinate_pair,[bing_lat,bing_lng],miles=True)
    except:
        dist=np.nan

    df['Bing_lat'][j]=bing_lat
    df['Bing_lng'][j]=bing_lng
    df['Bing_zip'][j]=bing_postal
    df['Bing_Address'][j]=bing_address
    df['Bing_return_dist'][j]=dist
    time.sleep(1)

    if j%33==1:
        time.sleep(0.1)
    if j%1000==999:
        time.sleep(1.2)
        print("i: "+str(i)+"|j: "+str(j)+" "+str(datetime.datetime.now()))

time.sleep(61)
print("Finished of i: "+str(i)+" "+str(datetime.datetime.now()))
df.to_csv("/home/jian/Projects/Smoothie_King/Rewards_Zip/SK_Unique_lat_lng_pair_JL_key_"+str(i)+"_"+str(datetime.datetime.now().date())+".csv",index=False)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

Finished of i: 0 2018-11-05 16:44:12.202861
